# GCC's ssc-cdi : ptychography pipeline for Cateretê v0.1
 
- #### Questions? E-mail gcc@lnls.br
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/wiki/

In [5]:
path = "/ibira/lnls/beamlines/caterete/proposals/20222104/data/ptycho3d/AA34_picro_crio2_03/scans/0078_AA34_picro_crio2_03_001.hdf5"

data = h5py.File(path,'r')['entry/data/data'][()]
data.shape

(425, 1, 3072, 3072)

## Imports

In [1]:
%matplotlib widget
import os, json, h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sscInstall
import sscCdi
import sscResolution

print(f'sscCdi version: {sscCdi.__version__}')
print(f'sscResolution version: {sscResolution.__version__}')

sscCdi version: 0.6.21
sscResolution version: 0.5.8


In [13]:
print([i for i in range(53,215)])

[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214]


## Inputs

In [2]:
input_dict = {
    
'beamline': 'CAT',
'detector': '540D',
'debug': False, # use False

"data_folder": "/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/data/ptycho3d/",
"acquisition_folders": ["glass_sphere"],
"projections": [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213],

"flatfield": "",

'CPUs': 160,
'GPUs': [0,1,2,3,4],

'binning': 1, 
'using_direct_beam': True, # if True, converts DP_center coordinates from raw to restored coordinates
'DP_center': [1433,1478],  # [center_y, center_x]
'fill_blanks': False, # interpolate blank lines after restoration
'keep_original_negative_values': False, 
'suspect_border_pixels': 3, # pixels
'detector_ROI_radius': 1350, # pixels

'position_rotation': -0.003, # radians. angle between detector and probe coordinate system

'object_padding': 50, # pixels

'incoherent_modes': 1, 
'probe_support': [ "circular", 150,0,0 ], # ["circular",radius_pxls,center_y, center_x]; (0,0) is the center of the image
'fresnel_number': -0.001,

"initial_obj": ["random"], # path to .npy, .hdf5, ["random"], ["constant"]
"initial_probe": ["inverse"], # path to .npy, .hdf5, ["inverse"], ["random"], ["constant"], ["circular",radius]

'Algorithm1': {'Batch': 64,
                'Beta': 0.995,
                'Epsilon': 0.01,
                'Iterations': 100,
                'Name': 'RAAR',
                'ProbeCycles': 4,
                'TV': 0},

'Algorithm2': {'Batch': 64,
                'Beta': 0.5,
                'Epsilon': 0.01,
                'Iterations': 50,
                'Name': 'RAAR',
                'ProbeCycles': 4,
                'TV': 0},
    
    
# 'Algorithm2': {'Batch': 64,
#                 'Epsilon': 0.01,
#                 'Iterations': 50,
#                 'Name': 'GL',
#                 'ObjBeta': 0.97,
#                 'ProbeBeta': 0.95,
#                 'TV': 0.0001},

# 'Algorithm3': {'Batch': 64,
#                 'Epsilon': 0.1,
#                 'Iterations': 50,
#                 'Name': 'GL',
#                 'ObjBeta': 0.5,
#                 'ProbeBeta': 0.5,
#                 'TV': 0.01},
}

input_dict = sscCdi.caterete.cat_ptycho_processing.define_paths(input_dict)

	Proposal path:  /ibira/lnls/labs/tepui/home/yuri.tonin/00000000/data/ptycho3d/
	Acquisition folder:  glass_sphere
	Output path: /ibira/lnls/labs/tepui/home/yuri.tonin/00000000/proc/recons/glass_sphere


## Load diffraction data to find center

In [3]:
# path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/scans/0000_SS03112022_02_001.hdf5" # enter path to raw diffractiom dataset

# fig, ax = plt.subplots(dpi=100)
# ax.imshow(np.squeeze(h5py.File(path,'r')['entry/data/data'])[0],norm=LogNorm(),cmap='jet')

# Run new job at cluster

In [4]:
server = sscInstall.connect_server()    

Username: Yuri.tonin


Password:


 ········


In [5]:
json_filepath_path = sscCdi.cat_ptycho_processing.save_input_dictionary(input_dict)
sscCdi.run_at_cluster(server,json_filepath_path,queue='dev-gcc',gpus=input_dict["GPUs"],cpus=input_dict["CPUs"],jobName='JOB',script_path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/ssc-cdi-0.6.17/bin/caterete_ptycho.py",slurm_path = '/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/inputs/')
json_filepath_path

Terminal output: Submitted batch job 57511



'/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/inputs/yuri.tonin_ptycho_input.json'


# Run Locally

## Restoration

In [ ]:
restoration_dict_list, restored_data_info_list = sscCdi.caterete.cat_restoration.restoration_ptycho_CAT(input_dict) # restoration of all frames; restored DPs saved at temporary folder

## Ptychography

In [10]:
input_dict

{'beamline': 'CAT',
 'detector': '540D',
 'debug': False,
 'data_folder': '/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/data/ptycho3d/',
 'acquisition_folders': ['glass_sphere'],
 'projections': [0],
 'flatfield': '/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/data/ptycho3d/glass_sphere/images/flat.hdf5',
 'CPUs': 64,
 'GPUs': [0, 1],
 'binning': 1,
 'using_direct_beam': True,
 'DP_center': [1433, 1478],
 'fill_blanks': False,
 'keep_original_negative_values': False,
 'suspect_border_pixels': 3,
 'detector_ROI_radius': 1350,
 'position_rotation': -0.003,
 'object_padding': 50,
 'incoherent_modes': 1,
 'probe_support': ['circular', 100, 0, 0],
 'fresnel_number': -0.001,
 'initial_obj': ['random'],
 'initial_probe': ['inverse'],
 'Algorithm1': {'Batch': 64,
  'Beta': 0.995,
  'Epsilon': 0.01,
  'Iterations': 100,
  'Name': 'RAAR',
  'ProbeCycles': 4,
  'TV': 0},
 'Algorithm2': {'Batch': 64,
  'Beta': 0.5,
  'Epsilon': 0.01,
  'Iterations': 50,
  'Name': 'RAAR',
  'ProbeCycles': 4,
 

In [ ]:
input_dict, obj, probe, probe_positions = sscCdi.caterete.cat_ptycho_processing.cat_ptychography(input_dict,restoration_dict_list,restored_data_info_list)
obj, probe, angles, positions, error = sscCdi.misc.save_volume_from_parts(input_dict)
sscCdi.misc.save_variable(input_dict, obj,name='object')
sscCdi.misc.save_variable(input_dict,probe,name='probe')
sscCdi.misc.save_variable(input_dict,positions,name='positions')
sscCdi.misc.save_variable(input_dict,angles,name='angles')
sscCdi.misc.save_variable(input_dict,error,name='error',group='log')
sscCdi.misc.save_json_logfile(input_dict) 
sscCdi.misc.delete_temporary_folders(input_dict)

In [ ]:
if 1: # phase
    preview = np.angle(obj)
else: # magnitude
    preview = np.abs(obj)

# sscCdi.misc.visualize_magnitude_and_phase(obj)
sscCdi.misc.deploy_visualizer(preview,title='Sinogram',cmap='gray')

In [ ]:
sscCdi.misc.deploy_visualizer(probe,title='Sinogram',cmap='gray')

## Crop object

In [9]:
input_dict['crop'] = [1400,2000,1300,1900]
cropped_object = sscCdi.caterete.cat_ptycho_processing.crop_sinogram(input_dict,obj,probe_positions)
if 1: # phase
    cropped_object = np.angle(cropped_object)
else: # magnitude
    cropped_object = np.abs(cropped_object)

In [10]:
sscCdi.misc.deploy_visualizer(cropped_object,title='Cropped sinogram',cmap='gray')

## Unwrap object

In [11]:
phase = sscCdi.caterete.unwrap_in_parallel(cropped_object)
sscCdi.misc.save_variable(input_dict,phase, name = 'object_unwrapped')

Using 32 parallel processes


100%|██████████| 1/1 [00:00<00:00,  5.47it/s]


In [12]:
sscCdi.misc.deploy_visualizer(phase,title='Unwrapped',cmap='gray')

## Fourier Ring Correlation

In [ ]:
frame = 0

if 1: # FRC on phase image
    img = phase[frame]
else: # else, on absorption image
    img = np.abs(cropped_object)[frame]

padding = 0
sharpness = 10
radius = 0.8

dic = sscResolution.frc.computep( sscResolution.frc.window( img, padding, [sharpness, radius] ) , input_dict["CPUs"] ) 
print(f"\tResolution via halfbit criterion: {1e9*input_dict['object_pixel']/dic['x']['H']:.2f} nm")

sscResolution.frc.plot(dic, {'label': "Resolution", 'unit': "nm", 'pxlsize': input_dict['object_pixel']*1e9} )